In [1]:
# conda install pytorch torchvision torchaudio -c pytorch
# pip install transformers
# pip install nltk
# pip install numpy 
# pip install pandas
#pip install wordcloud
# conda install matplotlib
# pip install scholarly
#pip install icecream

In [3]:
import textract
from transformers import pipeline
import nltk
import numpy as np
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud
from nltk.corpus import wordnet as wn

import urllib, urllib.request, urllib.parse
import xml.etree.ElementTree as ET
import re
import os
import os.path

from scholarly import scholarly
from icecream import ic

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Users/liam/anaconda3/envs/patents-ai/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/Users/liam/anaconda3/envs/patents-ai/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [4]:
# Preparatory steps
nltk.download('stopwords')
full_stop_words = set(nltk.corpus.stopwords.words('english'))

full_stop_words.add('b')
full_stop_words.add('c')
full_stop_words.add('d')
full_stop_words.add('e')
full_stop_words.add('f')
full_stop_words.add('g')
full_stop_words.add('h')
full_stop_words.add('j')
full_stop_words.add('k')
full_stop_words.add('l')
full_stop_words.add('m')
full_stop_words.add('n')
full_stop_words.add('p')
full_stop_words.add('q')
full_stop_words.add('r')
full_stop_words.add('u')
full_stop_words.add('v')
full_stop_words.add('x')
full_stop_words.add('w')
full_stop_words.add('y')
full_stop_words.add('z')
full_stop_words.add('pp')
full_stop_words.add('et')
full_stop_words.add('al')
full_stop_words.add('ha')
full_stop_words.add('li')
full_stop_words.add('sij')
full_stop_words.add('arxiv')

[nltk_data] Downloading package stopwords to /Users/liam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [318]:
# Common functions for parsing references from plain text

def re_matches(text, regex):
    iterator = re.findall(regex, text)

    refs = []
    for match in iterator:
        refs.append(match) 

    # print(len(refs))
    return refs

def re_digit():
    return r'\[\d*\]\ '

def re_first_first():
    return r'(?:(?:\-?[A-Z]\.\ )+(?:[A-Zvd][A-Za-z\-\u0080-\uFFFF]+\,?\ ?)+)'

def re_first_first_multiple():
    return '[^\,]\ (?:' + re_first_first() +'+and\ )?'

def re_first_last():
    return r'(?:(?:[A-Zvd][A-Za-z\-\u0080-\uFFFF]+)+\,\ (?:\-?[A-Z]\.\ ?)+\,?\ ?)'

def re_first_last_multiple():
    return '(?:' + re_first_last() +'+\,?and\ )'


def re_year():
    return r'\(?(?:\d{4})\)?'

def re_trailing_digit():
    return r'(?:[\d\ \,]+\ )?'


def re_in_journal():
    return r'(?:(?:in)?\ [^\,]+\,\ )'

def re_pages():
    return r'(?:\ (?:pp\.\ )?\d+\-?\d+)?'


# 3
# Example: A compact embedding for facial expression similarity.pdf
def gen_refs_with_pages_conjoined(text):
    r = ( 
        # re_digit() 
        r'('
        + re_first_first_multiple() 
        + re_first_first()  
        + r'\.[^\[]*?\,\ ' 
        + re_year() 
        + r'\.' 
        + re_trailing_digit() 
        + r')'
        )
    regex = re.compile(r)
    return re_matches(text, regex)

# 0
# Example: 2106.12139.pdf
def gen_refs_with_pages(text):
    regex = re.compile(r'\[\d*\]\ ((?:(?:\-?[A-Z]\.\ )+(?:[A-Zvd][A-Za-z]+\,?\ )+)+[^\[]*\(?(?:\d{4})\)?(?:\ pp\.\ \d+\-?\d+)?\.)')
    r = ( 
        re_digit() 
        + r'('
        + re_first_first()  
        + r'[^\[]*?\,?\ ' 
        + re_year() 
        + re_pages()
        + r')'
        + r'\.?'
        # + re_trailing_digit() 
        )
    regex = re.compile(r)
    return re_matches(text, regex)

# 1
# Example: Fashionista: A fashion-aware graphical system for exploring
def gen_refs_end_year_firstname_first(text):
    # regex = r'([A-Z]\.\ ?)+[A-Z][A-Za-z\-]+\,\ (?:.*\n.*){1,4}\,\ \d\d\d\d[a-b]?\.'
    r = ( 
        re_digit() 
        + r'('
        + re_first_first_multiple() 
        + re_first_first()  
        + r'[^\[]*?' 
        + re_in_journal()
        + re_year() 
        + r')'
        + r'\.?'
        )
    regex = re.compile(r)    
    return re_matches(text, regex) 

# 2
# Example: Fashion-gen: The generative fashion dataset and challenge.pdf
def gen_refs_end_year_firstname_last(text):
    regex = r'((?:(?:[A-Zvd][A-Za-zá\-\ ]+\,\ (?:\-?[A-Z]\.\ ?)+(?:\,\ )?)*)(?:\,?\ and\ )?(?:[A-Zvd][A-Za-zá\-\ ]+\,\ (?:\-?[A-Z]\.\ ?)+)[^\.]*\.\ [^\.]*\,\ \d{4}\.)'
    r = ( 
        r'('
        + re_first_last_multiple() 
        # + re_first_last()  
        + r'[^\[]*?\,?\ ' 
        # + re_pages()
        + re_year() 
        + r'\.'
        + r')'
        )
    regex = re.compile(r)    
    return re_matches(text, regex) 

# 2
# Example: Doubly Aligned Incomplete Multi-view Clustering
def gen_refs_text_brackets(text):
    r = ( 
        r'\[(?:\d+)\]\ '
        + r'('
        # + re_first_first_multiple() 
        # + re_first_first()  
        + r'[^\[]*?\,?\ ' 
        + re_year() 
        + r')'
        + r'\.?'
        )
    regex = re.compile(r)    
    return re_matches(text, regex) 

def gen_refs_end_year(text):
    regex = r'[A-Z][A-Za-z\-]+\,\ [A-Z]\.(?:.*\n.*){1,4}\,\ \d\d\d\d[a-b]?\.'
    return re_matches(text, regex) 

def gen_refs_end_year_with_text_brackets(text):
    regex = r'\[(?:[A-Za-z\ ]*\.?\,\ \d+)\]\ (?:.*\n.*){1,4}\d\d\d\d[a-b]?\.'
    return re_matches(text, regex) 

def gen_refs_end_year_with_number_brackets(text):
    regex = r'\[(?:\d+)\]\ (?:.*\n.*){1,4}\d\d\d\d[a-b]?\.'
    return re_matches(text, regex) 
    
def gen_refs_end_pages(text):
    text = text.replace('\n', ' ')
    regex = r'\[\d*\]\ [^[]*pp\.\ \d+\.\ '
    return re_matches(text, regex) 

def gen_refs_multiple_pass(text):
    refs = gen_refs_text_brackets(text)
    ref_type = 4
    if len(refs) == 0:
        refs = gen_refs_with_pages_conjoined(text)
        ref_type = 3
    if len(refs) == 0:
        refs = gen_refs_end_year_firstname_first(text)
        ref_type = 1
    if len(refs) == 0:
        refs = gen_refs_with_pages(text)
        ref_type = 0
    if len(refs) == 0:
        # refs = gen_refs_end_year(text)
        refs = gen_refs_end_year_firstname_last(text)
        ref_type = 2
    # if len(refs) == 0:
    #     refs = gen_refs_end_year_with_text_brackets(text)
    #     ref_type = 3
    if len(refs) == 0:
        refs = gen_refs_end_year_with_number_brackets(text)
        ref_type = 4
    if len(refs) == 0:
        refs = gen_refs_text_brackets(text)
        ref_type = 5    
    return refs, ref_type




In [289]:
# Common functions for extracting references parts

def make_dataframe(authors, title, journal, publisher, volume, year, pages, ref, source_file, source_title):
    return pd.DataFrame([[(', ').join(authors), title, journal, publisher, volume, year, pages, ref, source_file, source_title]], 
                        columns = ['authors', 'title', 'journal', 'publisher', 'volume', 'year', 'pages', 'full_ref', 'source_file', 'source_title'])

# Deprecated
def parse_ref00(ref, source_file, source_title):
    segments = ref.split(',')
    authors = []
    title = ''
    journal = ''
    publisher = ''
    volume = ''
    year = ''
    pages = ''
    finished_authors = False
    author_count = 0
    for count, value in enumerate(segments):
        if value.find('.') > -1 and finished_authors == False:
            authors.append(value.strip())
            author_count = author_count + 1
        elif value.find('.') == -1 and count < author_count + 2:
            if finished_authors == False:
                finished_authors = True
                title = value.strip()
            else:
                journal = value.strip()
        elif count == len(segments) - 1 and author_count + 2 == len(segments) - 1:
            match = re.search(r'(.*)\((\d\d\d\d)\)(.*)', value)
            if match is not None:
                volume = match.groups()[0]
                year = match.groups()[1]
                pages = match.groups()[2]
        # For when year and pages are comma-separated
        elif count == len(segments) - 2 and author_count + 3 == len(segments) - 1:
            year = value
        elif count == len(segments) - 1 and author_count + 3 == len(segments) - 1:
            pages = value
        # For when volume / publisher, year and pages are comma-separated
        elif count == len(segments) - 3 and author_count + 4 == len(segments) - 1:
            if value.isdigit():
                volume = value
            else:
                publisher = value
        elif count == len(segments) - 2 and author_count + 4 == len(segments) - 1:
            year = value
        elif count == len(segments) - 1 and author_count + 4 == len(segments) - 1:
            pages = value
    ic(authors)
    return make_dataframe(authors, title, journal, publisher, volume, year, pages, ref, source_file, source_title)

def _parse_ref_internals(ref, citation_pattern, authors_pattern, mapping = {
        'first_authors': 0,
        'last_author': 1,
        'title': 2,
        'journal': 3,
        'publisher': 4,
        'year': 5,
        'pages': 6
    }):
    all_authors = []
    title = ''
    journal = ''
    publisher = ''
    volume = ''
    year = ''
    pages = ''

    iterator = re.findall(citation_pattern, ref)
    for i in iterator:
        first_authors = i[mapping['first_authors']]
        last_author = i[mapping['last_author']].strip()
        title = i[mapping['title']]
        journal = i[mapping['journal']]
        publisher = i[mapping['publisher']]
        year = i[mapping['year']]
        pages = i[mapping['pages']]
        if first_authors != '':
            authors = re.findall(authors_pattern, first_authors)
            for a in authors:
                all_authors.append(a)
        all_authors.append(last_author)
    # ic(all_authors)
    # ic(title)
    # ic(journal)
    # ic(publisher)
    # ic(year)
    # ic(pages)
    return all_authors, title, journal, publisher, volume, year, pages

def parse_ref0(ref):
    citation_pattern = re.compile(
        r'((?:(?:\-?[A-Z]\.)+\ (?:[A-Zvd][A-Za-z\ \-]+\,\ ))*)' 
        + r'([^\,]*)\,\ ([^\,]*)\,\ (?:([^\,]*)\,\ )?(?:(\d+)\ )?' 
        + r'\(?(\d{4})\)?\,?\ ?' 
        + r'(?:pp\.\ )?(\d+\-?\d+)?')
    authors_pattern = re.compile(
        r'((?:\-?[A-Z]\.)+\ (?:[A-Zvd][A-Za-z\ \-]+))')

    return _parse_ref_internals(ref, citation_pattern, authors_pattern)

def parse_ref1(ref):
    citation_pattern = re.compile(
        r'((?:(?:\-?[A-Z]\.\ )+(?:[A-Zvd][A-Za-zá\-\ ]+\ ?)\,?\ )*)and\ ((?:\-?[A-Zvd]\.\ )+(?:[A-Zvd][A-Za-zá\-\ ]+\ ?)\,\ )' 
        + r'([^\,]*)\,.\ ([^\,\.]*)\,\ (?:([^\,]*)\,\ )?' 
        + r'(\d{4})\,?\ ?' 
        + r'(?:pp\.\ )?(\d+\-?\d+)?')
    authors_pattern = re.compile(r'(?:\-?[A-Z]\.\ )+[A-Zvd][A-Za-zá\-\ ]+')

    return _parse_ref_internals(ref, citation_pattern, authors_pattern)


def parse_ref2(ref):
    # citation_pattern = re.compile(r'((?:[A-Zvd][A-Za-zá\-\ ]+\,\ (?:\-?[A-Z]\.\ ?)+(?:\,\ )?)*)(?:\,?\ and\ )?([A-Zvd][A-Za-zá\-\ ]+\,\ (?:\-?[A-Z]\.\ ?)+)([^\.]*)\.\ (.*)\,\ (?:([^\,]*)\,\ )?(\d{4})\,?\ ?(?:pp\.\ )?(\d+\-?\d+)?')
    
    citation_pattern = re.compile(
        r'((?:(?:[A-Zvd][A-Za-z\-\u0080-\uFFFF]+)+\,\ (?:\-?[A-Z]\.\ ?)+(?:\,\ )?)*)(?:\,?\ and\ )?'
        + r'([A-Zvd][A-Za-za\-\u0080-\uFFFF]+\,\ (?:\-?[A-Z]\.\ ?)+)' 
        + r'([^\.]*)' # title
        + r'\.\ '
        + r'((?:[^\,]*))?'  # journal
        + r'(?:\,\ (.*(?:pp\.\ |d+\(d+\)\:)?\d+(?:.\d*)?))?' # pages 
        + r'(\.\ (?:[^\,]*))?' # publisher
        + r'\,\ '
        + r'(\d{4})'
        # + r'\.'
        )
    authors_pattern = re.compile(
        r'(?:[A-Zvd][A-Za-za\-\u0080-\uFFFF]+)+\,\ (?:\-?[A-Z]\.\ ?)+')

    return _parse_ref_internals(ref, citation_pattern, authors_pattern, mapping = {
        'first_authors': 0,
        'last_author': 1,
        'title': 2,
        'journal': 3,
        'pages': 4,
        'publisher': 5,
        'year': 6
    })

def parse_ref3(ref):
    citation_pattern = re.compile(
        r'((?:(?:\-?[A-Z]\.)+\ (?:[A-Zvd][A-Za-z\ \-]+\,\ ))*)and\ ((?:\-?[A-Z]\.)+\ (?:[A-Zvd][A-Za-z\ \-]+\.))?' 
        + r'([^\,]*)\,\ ([^\,]*)\,\ (?:([^\,]*)\,\ )?' 
        + r'(?:(\d+)\ )?' 
        + r'\(?(\d{4})\)?\,?\ ?' 
        + r'(?:pp\.\ )?(\d+\-?\d+)?')
    authors_pattern = re.compile(r'((?:\-?[A-Z]\.)+\ (?:[A-Zvd][A-Za-z\ \-]+))')

    return _parse_ref_internals(ref, citation_pattern, authors_pattern)



def parse_ref(ref, ref_type):
    if ref_type == 0:
        return parse_ref0(ref)
    elif ref_type == 1:
        return parse_ref1(ref)
    elif ref_type == 2:
        return parse_ref2(ref)
    elif ref_type == 3:
        return parse_ref3(ref)
    else:
        return parse_ref0(ref)


def parse_ref_and_makedataframe(ref, ref_type, source_file, source_title):
    authors, title, journal, publisher, volume, year, pages = parse_ref(ref, ref_type)
    return make_dataframe(authors, title, journal, publisher, volume, year, pages, ref, source_file, source_title)




In [7]:
# Other common functions

def get_lemma(word):
    return WordNetLemmatizer().lemmatize(word)

def remove_set_from_dict(s, d):
    for t in s:
        if t in d:
            del d[t]
    return d
    
def word_frequencies(text):
    words = re.findall('[A-Za-z][A-Za-z0-9]*', text)
    frequencies = {}
    for w in words:
        w = w.lower()
        w = get_lemma(w)
        if w not in full_stop_words:
            if w in frequencies:
                frequencies[w] += 1
            else:
                frequencies[w] = 1
    return frequencies

def word_frequencies_dist(text, frequencies):
    words = re.findall('[A-Za-z][A-Za-z0-9]*', text)
    for w in words:
        w = w.lower()
        w = get_lemma(w)
        if w not in full_stop_words:
            if w in frequencies:
                frequencies[w] += 1
            else:
                frequencies[w] = 1
    return frequencies



def wc(freqs, file_name):
    wordcloud = WordCloud(background_color="white", width=600, height=600, max_words=5000, contour_width=3, contour_color='steelblue')
    # Generate a word cloud
    #wordcloud.generate(long_string)
    wordcloud.generate_from_frequencies(freqs)
    wordcloud.to_file(f'data/word-cloud-{file_name}.png')
    wordcloud.to_image()



def arxiv_results(title):
    t =  urllib.parse.quote_plus(title)
    url = 'http://export.arxiv.org/api/query?search_query=all:'+t+'&start=0&max_results=1'
    data = urllib.request.urlopen(url)
    results = data.read().decode('utf-8')
    return results

def make_up_file_name(file_name):
    return 'refs/download/' +  file_name + '.pdf'

def extract_and_save_pdf_from_atom(atom_xml, title, title_no_colon, title_stem, pass_through=False):
    root = ET.fromstring(atom_xml)
    for e in root.findall('{http://www.w3.org/2005/Atom}entry'):
        t = e.find('{http://www.w3.org/2005/Atom}title')
        link = e.find('{http://www.w3.org/2005/Atom}link')
        if t.text.lower().find(title_stem.lower()) == 0:
            for link in e.findall('{http://www.w3.org/2005/Atom}link'):
                # Title must match and link must be a pdf
                if 'type' in link.attrib and link.attrib['type'] == 'application/pdf':
                    u  = link.attrib['href']
                    print(u, title)
                    if not pass_through:
                        urllib.request.urlretrieve(u, make_up_file_name(title))

def test_element(atom_xml):
    root = ET.fromstring(atom_xml)
    entry = root.find('{http://www.w3.org/2005/Atom}entry')
    if entry is not None:
        return True
    else:
        return False

def top_dist(freqs, n):
    counter = 0
    for w in sorted(freqs, key = freqs.get, reverse = True):
        counter = counter + 1
        print(w, freqs[w])
        if counter == n:
            break

In [8]:
file_name = '2106.12139.pdf'
title = 'PatentNet: A Large-Scale Incomplete Multiview, Multimodal, Multilabel Industrial Goods Image Database'


text = textract.process("refs/" + file_name).decode('utf-8')
freqs = word_frequencies(text)

references = pd.DataFrame(columns = ['authors', 'title', 'journal', 'publisher', 'volume', 'year', 'pages', 'full_ref', 'source_file', 'source_title'])

refs = gen_refs_multiple_pass(text)
for r in refs:
    r = r.replace('\n', ' ')
    references = references.append(parse_ref_and_makedataframe(r, 0, file_name, title))

references.to_csv('data/references.csv')



AttributeError: 'list' object has no attribute 'replace'

In [59]:
# Generate the word cloud from the seed file frequencies.
wc(freqs, file_name)



In [60]:
# Test code

# t = references['title'].iloc[7]
# print(t)
# r = arxiv_results(t.strip())
# print(r)
# # ET.tostring(r)

In [56]:
# Attempt to locate references on Arxiv, and save
counter = 0
for title in references['title']:
    f = make_up_file_name(title)
    counter = counter + 1
    if not os.path.isfile(f):
        r = arxiv_results(title)
        title_stem = title
        title_no_colon = title
        if title.find(':') > -1:
            title_stem = title[:title.index(':')]
            title_no_colon = title[:title.index(':')] + title[title.index(':')+1:]
        
        has_entry = test_element(r)

        # Remove the semi-colon - seems to confuse Arxiv API
        if not has_entry:
            r = arxiv_results(title_no_colon)
            has_entry = test_element(r)

        # Remove everything after the semi-colon
        if not has_entry:
            r = arxiv_results(title_stem)
            has_entry = test_element(r)

        print(counter, title)
        if has_entry:
            extract_and_save_pdf_from_atom(r, title, title_no_colon, title_stem, False)



1 Deep learning for visual understanding: A review
2 Deep convolutional neural networks for image classification: A comprehensive review
4 Microsoft coco: Common objects in context
5 Deepfashion: Powering robust clothes recognition and retrieval with rich annotations
http://arxiv.org/pdf/1901.07973v1 Deepfashion: Powering robust clothes recognition and retrieval with rich annotations


KeyboardInterrupt: 

In [320]:

def test_ref_type(file):
    f = os.path.join(file)
    text = textract.process(f).decode('utf-8')
    text = text.replace('\n', ' ')
    # regex = r'(?:[A-Zvd][A-Za-zá\-]+)\,\ (?:[A-Z]\.\ ?)+'
    # regex = r'((?:(?:[A-Zvd][A-Za-zá\-\ ]+\,\ (?:\-?[A-Z]\.\ ?)+(?:\,\ )?)*)(?:\,?\ and\ )?(?:[A-Zvd][A-Za-zá\-\ ]+\,\ (?:\-?[A-Z]\.\ ?)+)[^\.]*\.\ [^\.]*\,\ \d{4}\.)'
    # pattern = re.compile(regex)
    # iterator = re.findall(pattern, text)
    # ic(file)
    # for i in iterator:
    #     ic(i)
    refs_local, ref_type = gen_refs_multiple_pass(text)
    # refs_local = gen_refs_text_brackets(text)
    # ref_type = 1
    ic(file)
    ic(ref_type)
    ic(len(refs_local))
    # ic(refs_local)
    # ic(text)



test_ref_type('./refs/2106.12139.pdf')
test_ref_type('./refs/download/A compact embedding for facial expression similarity.pdf')
test_ref_type('./refs/download/Central similarity quantization for efficient image and video retrieval.pdf')
test_ref_type('./refs/download/Doubly Aligned Incomplete Multi-view Clustering.pdf')
test_ref_type('./refs/download/Efficientnet: Rethinking model scaling for convolutional neural networks.pdf')
test_ref_type('./refs/download/Fashion-gen: The generative fashion dataset and challenge.pdf')
test_ref_type('./refs/download/Fashion-mnist: a novel image dataset for benchmarking machine learning algorithms.pdf')
test_ref_type('./refs/download/Fashionista: A fashion-aware graphical system for exploring visually similar items.pdf')
test_ref_type('./refs/download/Hashnet: Deep learning to hash by continuation.pdf')
test_ref_type('./refs/download/Imagenet: A large-scale hierarchical image database.pdf')
test_ref_type('./refs/download/Partnet: A large-scale benchmark for finegrained and hierarchical part-level 3d object understanding.pdf')
test_ref_type('./refs/download/Shapenet: An information-rich 3d model repository.pdf')





ic| file: './refs/2106.12139.pdf'
ic| ref_type: 4
ic| len(refs_local): 41
ic| file: './refs/download/A compact embedding for facial expression similarity.pdf'
ic| ref_type: 4
ic| len(refs_local): 36
ic| file: ('./refs/download/Central similarity quantization for efficient image and '
           'video retrieval.pdf')
ic| ref_type: 4
ic| len(refs_local): 31
ic| file: './refs/download/Doubly Aligned Incomplete Multi-view Clustering.pdf'
ic| ref_type: 5
ic| len(refs_local): 0
ic| file: ('./refs/download/Efficientnet: Rethinking model scaling for convolutional '
           'neural networks.pdf')
ic| ref_type: 2
ic| len(refs_local): 50
ic| file: './refs/download/Fashion-gen: The generative fashion dataset and challenge.pdf'
ic| ref_type: 2
ic| len(refs_local): 29
ic| file: ('./refs/download/Fashion-mnist: a novel image dataset for benchmarking '
           'machine learning algorithms.pdf')
ic| ref_type: 4
ic| len(refs_local): 2
ic| file: ('./refs/download/Fashionista: A fashion-aware graph

In [321]:
d = './refs/download'

# d = './refs'
all_freqs = {**freqs}
all_references = references.copy()
for file in os.listdir(d):
    # if file.endswith("Fashionista: A fashion-aware graphical system for exploring visually similar items.pdf"):
    # if file.endswith("A compact embedding for facial expression similarity.pdf"):
    # if file.endswith("Fashion-gen: The generative fashion dataset and challenge.pdf"):
    # if file.endswith("2106.12139.pdf"):
    if file.endswith(".pdf") and file.find('Deepfashion') == -1:
        f = os.path.join(d, file)
        ic(f)
        try:
            text_local = textract.process(f).decode('utf-8')
            text_local = text_local.replace('\n', ' ')
            all_freqs = word_frequencies_dist(text_local, all_freqs)
            refs_local, ref_type  = gen_refs_multiple_pass(text_local)

            ic(len(refs_local))
            ic(ref_type)
            counter = 0
            for r in refs_local:
                counter = counter + 1
                # if counter > 3:
                #     break
                # ic(r)

                # authors, title, journal, publisher, volume, year, pages = parse_ref(r, ref_type)
                # ic(authors)
                # ic(title)
                # ic(journal)
                # ic(publisher)
                # ic(volume)
                # ic(year)
                # ic(pages)

                all_references = all_references.append(parse_ref_and_makedataframe(r, ref_type, f, ''))

        except Exception as e:
            ic(e)
    

ic| f: ('./refs/download/Fashionista: A fashion-aware graphical system for exploring '
        'visually similar items.pdf')
ic| len(refs_local): 8
ic| ref_type: 4
ic| f: './refs/download/Fashion-gen: The generative fashion dataset and challenge.pdf'
ic| len(refs_local): 29
ic| ref_type: 2
ic| f: ('./refs/download/Central similarity quantization for efficient image and '
        'video retrieval.pdf')
ic| len(refs_local): 31
ic| ref_type: 4
ic| f: './refs/download/Doubly Aligned Incomplete Multi-view Clustering.pdf'
ic| len(refs_local): 0
ic| ref_type: 5
ic| f: ('./refs/download/Partnet: A large-scale benchmark for finegrained and '
        'hierarchical part-level 3d object understanding.pdf')
ic| len(refs_local): 46
ic| ref_type: 4
ic| f: ('./refs/download/Efficientnet: Rethinking model scaling for convolutional '
        'neural networks.pdf')
ic| len(refs_local): 50
ic| ref_type: 2
ic| f: './refs/download/A compact embedding for facial expression similarity.pdf'
ic| len(refs_local)

In [322]:
all_references.to_csv('data/all_references.csv')

In [185]:
# # f = './refs/download/Shapenet: An information-rich 3d model repository.pdf'
# f = './refs/download/Fashion-gen: The generative fashion dataset and challenge.pdf'
# # f = './refs/download/Doubly Aligned Incomplete Multi-view Clustering.pdf'
# # f = './refs/download/Shapenet: An information-rich 3d model repository.pdf'

# t = textract.process(f).decode('utf-8')
# print(t)
# # t = t.replace('\n', ' ')
# # regex = r'\[\d*\]\ [^\n]*\n[^\n]*\d\d\d\d\.'
# # regex = r'[A-Z][A-Za-z\-]+\,(.*\n.*){0,5}\,\ \d\d\d\d\.'
# # regex = r'[A-Z][A-Za-z\-]+\,\ [A-Z]\.(?:.*\n.*){1,4}\,\ \d\d\d\d[a-b]?\.'
# # regex = r'\ [A-Z][A-Za-z\,\.\ \-\n]+[A-Z]\.\ [A-Z](?:.*\n.*){0,5}\,\ \d\d\d\d\.'
# # regex = r'[A-Z][A-Za-z\,\.\ ]+[A-Z]\.\ [A-Z](.\n.)*\1{5}.*\d\d\d\d\.\ '
# # regex = r'\[(?:[A-Za-z\ \.]*\,\ \d+)\]\ (?:.*\n.*){1,4}\d\d\d\d[a-b]?\.'
# regex = r'\[(?:\d+)\]\ (?:.*\n.*){1,4}\d\d\d\d[a-b]?\.'
# pattern = re.compile(regex)
# iterator = re.findall(pattern, t)

# print(len(iterator))
# refs = []
# for match in iterator:
#     refs.append(match) 
# print(refs)

In [52]:
wc(all_freqs, 'all_freqs')



In [26]:
s = """Liu, Z., Luo, P., Wang, X., and Tang, X. Deep learning
face attributes in the wild. In Proceedings of the IEEE
International Conference on Computer Vision, pp. 3730–
3738, 2015."""

# \1{5}.*\d\d\d\d\.
# regex = r'[A-Z][A-Za-z\,\.\ ]+[A-Z]\.\ [A-Z](?:.*\n.*){0,5}\,\ \d\d\d\d\.'
regex = r'[A-Z][A-Za-z\-]+\,\ [A-Z]\.(?:.*\n.*){0,5}\,\ \d\d\d\d\.'
pattern = re.compile(regex)
iterator = re.findall(pattern, s)
print(iterator)


['Liu, Z., Luo, P., Wang, X., and Tang, X. Deep learning\nface attributes in the wild. In Proceedings of the IEEE\nInternational Conference on Computer Vision, pp. 3730–\n3738, 2015.']


In [121]:
top_dist(freqs, 5)

image 95
industrial 92
good 81
view 67
patentnet 61


In [58]:
top_dist(all_freqs, 1000)

image 495
model 309
part 283
hash 280
method 267
learning 262
category 261
dataset 256
shape 249
data 247
network 240
view 207
object 190
deep 187
segmentation 185
figure 181
annotation 177
center 162
training 154
different 153
code 149
using 146
retrieval 135
set 131
result 130
similarity 129
hashnet 129
scale 127
large 126
function 126
table 126
level 123
imagenet 122
fashion 122
multi 122
accuracy 119
instance 118
loss 116
label 115
recognition 114
information 110
datasets 110
number 109
bit 109
expression 107
ieee 105
triplet 105
use 103
fine 101
two 100
depth 100
based 99
three 99
visual 98
semantic 98
class 97
computer 97
efficientnet 97
performance 96
vision 96
grained 95
scaling 95
hashing 95
hierarchical 94
page 94
feature 93
also 93
point 93
embedding 93
industrial 92
good 92
similar 92
one 92
matrix 91
item 89
work 88
conference 88
neural 87
space 86
example 85
algorithm 85
classification 84
show 83
proposed 82
incomplete 80
top 79
used 78
clustering 78
first 76
shapenet 76
